In [1]:
import pandas as pd
import os
import pymysql

host ='localhost'
port='3306'
user='root'
password='zhuyangjian22'
database='zhuyangjian'


In [2]:
conn=pymysql.connect(host=host,port=int(port), user=user, passwd=password, db=database, charset='utf8mb4')
datasets=pd.read_sql_query('select houjin_num,houjin_name,address from kihon1',conn)
datasets.head(10)

E:\anaconda\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,houjin_num,houjin_name,address
0,1000011000005,国立国会図書館,東京都千代田区永田町１丁目１０－１
1,1000012010003,内閣法制局,東京都千代田区霞が関３丁目１－１中央合同庁舎第４号館
2,1000012010011,郵政民営化推進本部,東京都千代田区永田町１丁目１１－３９
3,1000012010028,国土強靱化推進本部,東京都千代田区永田町１丁目６－１
4,1000012010036,特定複合観光施設区域整備推進本部,東京都千代田区霞が関３丁目２－５霞が関ビルディング１２階
5,1000012020002,公害等調整委員会,東京都千代田区霞が関３丁目１－１中央合同庁舎第４号館１０階
6,1000012030001,法務省,東京都千代田区霞が関１丁目１－１
7,1000012090004,中小企業庁,東京都千代田区霞が関１丁目３－１
8,1000012100002,運輸安全委員会,東京都新宿区四谷１丁目６－１
9,1000012110001,環境省,東京都千代田区霞が関１丁目２－２中央合同庁舎第５号館


In [3]:
import urllib
import urllib.request as request
import pandas as pd
import json



In [41]:
url='https://map.yahooapis.jp/search/local/V1/localSearch?appid='
api_key='dj00aiZpPTVjajlkMUNnOHVDayZzPWNvbnN1bWVyc2VjcmV0Jng9ODA-'
temp4=pd.DataFrame()
temp6=pd.DataFrame()
temp8=pd.DataFrame()
for name, row in datasets.iterrows(): #iterrows 返回行索引
    try:
        #print(row)
        url2=url+api_key+"&query="+urllib.parse.quote(row.houjin_name)+"&output=json" #parse.queto 将字符转换为url的安全字符
        #print(url2)
        response=request.urlopen(url2)
        contents=response.read()
        temp2=json.loads(contents.decode())
        response.close()
        temp5=pd.json_normalize(temp2)
        print(name)
        if (int(temp5["ResultInfo.Count"])>0):
            try:
                a=name
                print(url2)
                temp5=pd.json_normalize(temp2,"Feature")
                temp5['name']=a #变更name列的值
                temp5['original_address']=row.address
            
            #以上先按feature 展开
                temp3=pd.json_normalize(temp2,["Feature",["Property","Genre"]])
                temp3["name"]=a
                temp3["original_address"]=row.address
            #将feature 展开后，提出property 的genre的数据到第一次元
                station=pd.json_normalize(temp2,["Feature",["Property","Station"]])
                station["name"]=a
                station["original_address"]=row.address
            #读取station 的数据
           
                print(row.houjin_name)
           
                temp6=pd.concat([temp6,temp5],axis=0)
                temp4=pd.concat([temp4,temp3],axis=0)
                temp8=pd.concat([temp8,station],axis=0)
            except KeyError as e:
                print('1')
            
                
    except urllib.error.HTTPError:
        print('not')

0
https://map.yahooapis.jp/search/local/V1/localSearch?appid=dj00aiZpPTVjajlkMUNnOHVDayZzPWNvbnN1bWVyc2VjcmV0Jng9ODA-&query=%E5%9B%BD%E7%AB%8B%E5%9B%BD%E4%BC%9A%E5%9B%B3%E6%9B%B8%E9%A4%A8&output=json
国立国会図書館
1
https://map.yahooapis.jp/search/local/V1/localSearch?appid=dj00aiZpPTVjajlkMUNnOHVDayZzPWNvbnN1bWVyc2VjcmV0Jng9ODA-&query=%E5%86%85%E9%96%A3%E6%B3%95%E5%88%B6%E5%B1%80&output=json
内閣法制局
2
3
4
5
https://map.yahooapis.jp/search/local/V1/localSearch?appid=dj00aiZpPTVjajlkMUNnOHVDayZzPWNvbnN1bWVyc2VjcmV0Jng9ODA-&query=%E5%85%AC%E5%AE%B3%E7%AD%89%E8%AA%BF%E6%95%B4%E5%A7%94%E5%93%A1%E4%BC%9A&output=json
公害等調整委員会
6
https://map.yahooapis.jp/search/local/V1/localSearch?appid=dj00aiZpPTVjajlkMUNnOHVDayZzPWNvbnN1bWVyc2VjcmV0Jng9ODA-&query=%E6%B3%95%E5%8B%99%E7%9C%81&output=json
法務省
7
https://map.yahooapis.jp/search/local/V1/localSearch?appid=dj00aiZpPTVjajlkMUNnOHVDayZzPWNvbnN1bWVyc2VjcmV0Jng9ODA-&query=%E4%B8%AD%E5%B0%8F%E4%BC%81%E6%A5%AD%E5%BA%81&output=json
中小企業庁
8
https://map.yahooapis.

47
https://map.yahooapis.jp/search/local/V1/localSearch?appid=dj00aiZpPTVjajlkMUNnOHVDayZzPWNvbnN1bWVyc2VjcmV0Jng9ODA-&query=%E9%AB%98%E7%9F%A5%E5%AE%B6%E5%BA%AD%E8%A3%81%E5%88%A4%E6%89%80&output=json
高知家庭裁判所
48
https://map.yahooapis.jp/search/local/V1/localSearch?appid=dj00aiZpPTVjajlkMUNnOHVDayZzPWNvbnN1bWVyc2VjcmV0Jng9ODA-&query=%E6%AD%A6%E8%94%B5%E9%87%8E%E7%B0%A1%E6%98%93%E8%A3%81%E5%88%A4%E6%89%80&output=json
武蔵野簡易裁判所
49
https://map.yahooapis.jp/search/local/V1/localSearch?appid=dj00aiZpPTVjajlkMUNnOHVDayZzPWNvbnN1bWVyc2VjcmV0Jng9ODA-&query=%E8%97%A4%E6%B2%A2%E7%B0%A1%E6%98%93%E8%A3%81%E5%88%A4%E6%89%80&output=json
藤沢簡易裁判所
50
https://map.yahooapis.jp/search/local/V1/localSearch?appid=dj00aiZpPTVjajlkMUNnOHVDayZzPWNvbnN1bWVyc2VjcmV0Jng9ODA-&query=%E5%A4%A7%E5%AE%AE%E7%B0%A1%E6%98%93%E8%A3%81%E5%88%A4%E6%89%80&output=json
大宮簡易裁判所
51
https://map.yahooapis.jp/search/local/V1/localSearch?appid=dj00aiZpPTVjajlkMUNnOHVDayZzPWNvbnN1bWVyc2VjcmV0Jng9ODA-&query=%E7%A7%A9%E7%88%B6%E7%B0%A1%E

91
https://map.yahooapis.jp/search/local/V1/localSearch?appid=dj00aiZpPTVjajlkMUNnOHVDayZzPWNvbnN1bWVyc2VjcmV0Jng9ODA-&query=%E5%85%AB%E9%9B%B2%E7%B0%A1%E6%98%93%E8%A3%81%E5%88%A4%E6%89%80&output=json
八雲簡易裁判所
92
https://map.yahooapis.jp/search/local/V1/localSearch?appid=dj00aiZpPTVjajlkMUNnOHVDayZzPWNvbnN1bWVyc2VjcmV0Jng9ODA-&query=%E7%95%99%E8%90%8C%E7%B0%A1%E6%98%93%E8%A3%81%E5%88%A4%E6%89%80&output=json
留萌簡易裁判所
93
https://map.yahooapis.jp/search/local/V1/localSearch?appid=dj00aiZpPTVjajlkMUNnOHVDayZzPWNvbnN1bWVyc2VjcmV0Jng9ODA-&query=%E9%81%A0%E8%BB%BD%E7%B0%A1%E6%98%93%E8%A3%81%E5%88%A4%E6%89%80&output=json
遠軽簡易裁判所
94
https://map.yahooapis.jp/search/local/V1/localSearch?appid=dj00aiZpPTVjajlkMUNnOHVDayZzPWNvbnN1bWVyc2VjcmV0Jng9ODA-&query=%E5%AE%89%E8%8A%B8%E7%B0%A1%E6%98%93%E8%A3%81%E5%88%A4%E6%89%80&output=json
安芸簡易裁判所
95
https://map.yahooapis.jp/search/local/V1/localSearch?appid=dj00aiZpPTVjajlkMUNnOHVDayZzPWNvbnN1bWVyc2VjcmV0Jng9ODA-&query=%E4%BB%8A%E6%B2%BB%E7%B0%A1%E6%98%93%E8

135
https://map.yahooapis.jp/search/local/V1/localSearch?appid=dj00aiZpPTVjajlkMUNnOHVDayZzPWNvbnN1bWVyc2VjcmV0Jng9ODA-&query=%E9%9A%8E%E4%B8%8A%E7%94%BA&output=json
階上町
136
137
https://map.yahooapis.jp/search/local/V1/localSearch?appid=dj00aiZpPTVjajlkMUNnOHVDayZzPWNvbnN1bWVyc2VjcmV0Jng9ODA-&query=%E4%B8%AD%E9%83%A8%E4%B8%8A%E5%8C%97%E5%BA%83%E5%9F%9F%E4%BA%8B%E6%A5%AD%E7%B5%84%E5%90%88&output=json
中部上北広域事業組合
138
139
https://map.yahooapis.jp/search/local/V1/localSearch?appid=dj00aiZpPTVjajlkMUNnOHVDayZzPWNvbnN1bWVyc2VjcmV0Jng9ODA-&query=%E5%BC%98%E5%89%8D%E5%9C%B0%E5%8C%BA%E7%92%B0%E5%A2%83%E6%95%B4%E5%82%99%E4%BA%8B%E5%8B%99%E7%B5%84%E5%90%88&output=json
弘前地区環境整備事務組合
140
https://map.yahooapis.jp/search/local/V1/localSearch?appid=dj00aiZpPTVjajlkMUNnOHVDayZzPWNvbnN1bWVyc2VjcmV0Jng9ODA-&query=%E9%BB%92%E7%9F%B3%E5%9C%B0%E5%8C%BA%E6%B8%85%E6%8E%83%E6%96%BD%E8%A8%AD%E7%B5%84%E5%90%88&output=json
黒石地区清掃施設組合
141
https://map.yahooapis.jp/search/local/V1/localSearch?appid=dj00aiZpPTVjajlkMUN

191
192
193
https://map.yahooapis.jp/search/local/V1/localSearch?appid=dj00aiZpPTVjajlkMUNnOHVDayZzPWNvbnN1bWVyc2VjcmV0Jng9ODA-&query=%E5%A1%A9%E8%B0%B7%E7%94%BA&output=json
塩谷町
194
195
https://map.yahooapis.jp/search/local/V1/localSearch?appid=dj00aiZpPTVjajlkMUNnOHVDayZzPWNvbnN1bWVyc2VjcmV0Jng9ODA-&query=%E7%8E%89%E6%9D%91%E7%94%BA&output=json
玉村町
196
https://map.yahooapis.jp/search/local/V1/localSearch?appid=dj00aiZpPTVjajlkMUNnOHVDayZzPWNvbnN1bWVyc2VjcmV0Jng9ODA-&query=%E5%AF%8C%E5%B2%A1%E7%94%98%E6%A5%BD%E8%A1%9B%E7%94%9F%E6%96%BD%E8%A8%AD%E7%B5%84%E5%90%88&output=json
富岡甘楽衛生施設組合
197
https://map.yahooapis.jp/search/local/V1/localSearch?appid=dj00aiZpPTVjajlkMUNnOHVDayZzPWNvbnN1bWVyc2VjcmV0Jng9ODA-&query=%E7%94%98%E6%A5%BD%E8%A5%BF%E9%83%A8%E7%92%B0%E5%A2%83%E8%A1%9B%E7%94%9F%E6%96%BD%E8%A8%AD%E7%B5%84%E5%90%88&output=json
甘楽西部環境衛生施設組合
198
199
https://map.yahooapis.jp/search/local/V1/localSearch?appid=dj00aiZpPTVjajlkMUNnOHVDayZzPWNvbnN1bWVyc2VjcmV0Jng9ODA-&query=%E5%A4%9A%E9%87%8E

URLError: <urlopen error [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。>

In [42]:
temp4

,Code,Name,name,original_address
0,0423,公共サービス、各種団体,0,東京都千代田区永田町１丁目１０－１
1,0422001,国の機関,0,東京都千代田区永田町１丁目１０－１
2,0423,公共サービス、各種団体,0,東京都千代田区永田町１丁目１０－１
3,0423,公共サービス、各種団体,0,東京都千代田区永田町１丁目１０－１
4,0422001,国の機関,0,東京都千代田区永田町１丁目１０－１
...,...,...,...,...
3,0508014,医師会,239,東京都東村山市本町１丁目２番地３
4,0505006,建設、工事、不動産（その他）,239,東京都東村山市本町１丁目２番地３
5,0305007,公園,239,東京都東村山市本町１丁目２番地３
6,0422003,市区町村機関,239,東京都東村山市本町１丁目２番地３


In [43]:
temp8

,Id,SubId,Name,Railway,Exit,ExitId,Distance,Time,Geometry.Type,Geometry.Coordinates,name,original_address
0,22856,2285601,永田町,東京メトロ南北線/東京メトロ半蔵門線/東京メトロ有楽町線,1,4797,388,4,NaN,NaN,0,東京都千代田区永田町１丁目１０－１
1,22856,2285601,永田町,東京メトロ南北線/東京メトロ半蔵門線/東京メトロ有楽町線,2,4798,402,4,NaN,NaN,0,東京都千代田区永田町１丁目１０－１
2,22856,2285601,永田町,東京メトロ南北線/東京メトロ半蔵門線/東京メトロ有楽町線,3,4800,628,7,NaN,NaN,0,東京都千代田区永田町１丁目１０－１
3,22596,2259601,霞ケ関(東京都),東京メトロ丸ノ内線/東京メトロ千代田線/東京メトロ日比谷線,B1a,3672,261,3,point,"139.753784,35.673923",0,東京都千代田区永田町１丁目１０－１
4,22698,2269801,桜田門,東京メトロ有楽町線,5,4022,416,5,point,"139.753394,35.677270",0,東京都千代田区永田町１丁目１０－１
...,...,...,...,...,...,...,...,...,...,...,...,...
23,22728,2272801,新秋津,JR在来線,出口,4119,836,10,NaN,NaN,239,東京都東村山市本町１丁目２番地３
24,22491,2249101,秋津,西武鉄道,北口,3081,859,10,NaN,NaN,239,東京都東村山市本町１丁目２番地３
25,23028,2302801,八坂,西武鉄道,出口,5330,528,6,NaN,NaN,239,東京都東村山市本町１丁目２番地３
26,22896,2289601,萩山,西武鉄道,南口,4927,957,11,NaN,NaN,239,東京都東村山市本町１丁目２番地３


In [64]:
temp6['Property.Tel1']

0         03-3506-3300
1         03-3580-4111
2         0774-98-1200
3    03-6737-9011,9014
4         0774-98-1200
           ...        
5         042-392-1392
6         042-394-6333
7         042-397-1212
8         042-392-6388
9         042-390-2211
Name: Property.Tel1, Length: 1227, dtype: object

In [45]:
temp10=pd.merge(temp6,temp4,on="name",how="inner")
temp11=pd.merge(temp10,temp8,on="name",how="inner")

In [65]:
temp11["Property.Tel1"]

0         03-3506-3300
1         03-3506-3300
2         03-3506-3300
3         03-3506-3300
4         03-3506-3300
              ...     
116846    042-390-2211
116847    042-390-2211
116848    042-390-2211
116849    042-390-2211
116850    042-390-2211
Name: Property.Tel1, Length: 116851, dtype: object

In [70]:
name2=temp11.name
name=temp11.Name_x
station_name=temp11.Name
station_railway=temp11.Railway
station_distance=temp11.Distance
station_exit=temp11.Exit
address2=temp11.original_address
category=temp11["Name_y"]
govcode=temp11["Property.GovernmentCode"]
tel=temp11["Property.Tel1"]
yahoo_list=pd.concat([name2,name,station_name,station_railway,station_distance,station_exit,address2,category,govcode,tel],axis=1)


In [71]:
yahoo_list=yahoo_list.drop_duplicates()
yahoo_list.set_index("name")

,Name_x,Name,Railway,Distance,Exit,original_address,Name_y,Property.GovernmentCode,Property.Tel1
name,,,,,,,,,
0,国立国会図書館利用案内,永田町,東京メトロ南北線/東京メトロ半蔵門線/東京メトロ有楽町線,388,1,東京都千代田区永田町１丁目１０－１,公共サービス、各種団体,13101,03-3506-3300
0,国立国会図書館利用案内,永田町,東京メトロ南北線/東京メトロ半蔵門線/東京メトロ有楽町線,402,2,東京都千代田区永田町１丁目１０－１,公共サービス、各種団体,13101,03-3506-3300
0,国立国会図書館利用案内,永田町,東京メトロ南北線/東京メトロ半蔵門線/東京メトロ有楽町線,628,3,東京都千代田区永田町１丁目１０－１,公共サービス、各種団体,13101,03-3506-3300
0,国立国会図書館利用案内,霞ケ関(東京都),東京メトロ丸ノ内線/東京メトロ千代田線/東京メトロ日比谷線,261,B1a,東京都千代田区永田町１丁目１０－１,公共サービス、各種団体,13101,03-3506-3300
0,国立国会図書館利用案内,桜田門,東京メトロ有楽町線,416,5,東京都千代田区永田町１丁目１０－１,公共サービス、各種団体,13101,03-3506-3300
...,...,...,...,...,...,...,...,...,...
239,東村山市南部地域包括支援センター,新秋津,JR在来線,836,出口,東京都東村山市本町１丁目２番地３,市区町村機関,13213,042-390-2211
239,東村山市南部地域包括支援センター,秋津,西武鉄道,859,北口,東京都東村山市本町１丁目２番地３,市区町村機関,13213,042-390-2211
239,東村山市南部地域包括支援センター,八坂,西武鉄道,528,出口,東京都東村山市本町１丁目２番地３,市区町村機関,13213,042-390-2211
